This script is designed to process text files in a structured manner. It begins by loading the necessary data and defining agents and tools required for the task. These agents and tools are configured to work sequentially, ensuring that the text files located in the root directory are properly loaded. Once the text files are loaded, the script processes their content by resuming (extracting key points) from each file. Finally, it generates a concise summary of all the texts, providing a high-level overview of the information contained in the files.

In [1]:
import sys
import os
from dotenv import load_dotenv

src_path = os.path.abspath(os.path.join(os.getcwd(), "..", "src"))
print("Adding src folder to path:", src_path)
sys.path.insert(0, src_path)

load_dotenv(f"{src_path}/.env")

Adding src folder to path: c:\Users\ricar\Github\contractor\src


True

### Loading Models from the Source Folder

To begin, ensure that the required models are loaded from the source folder. This step is crucial for initializing the necessary components for processing and analysis. Verify that the source folder contains all the relevant model files before proceeding.

In [2]:
from app.schemas.models import VideoData, AudioData, ImageData, TextData

### Loading Data Models

In the next cell, we will load the data models corresponding to various data sources: audio, text, video, and image. These models represent the structured data from the respective folders inside the `./data` directory. Each model is designed to handle the specific characteristics of its data type, ensuring efficient processing and analysis.

In [3]:
audio_data_1 = AudioData(
    source="./data/audios/azure-podcast-2.mp3",
    objective="Explains Azure ACA and its features",
    tags=["azure", "aca", "containers"]
)

audio_data_2 = AudioData(
    source="./data/audios/azure-podcast-2.mp3",
    objective="Explains Azure in general",
    tags=["azure", "definitions"]
)

print("AudioData instance:", audio_data_1)
print("AudioData instance:", audio_data_2)

AudioData instance: source='./data/audios/azure-podcast-2.mp3' objective='Explains Azure ACA and its features' tags=['azure', 'aca', 'containers'] encoding=None embeddings=None
AudioData instance: source='./data/audios/azure-podcast-2.mp3' objective='Explains Azure in general' tags=['azure', 'definitions'] encoding=None embeddings=None


In [4]:
text_data_1 = TextData(
    source="./data/documents/aoai-assistants.pdf",
    objective="Explains Azure OpenAI and its features",
    tags=["azure", "aoai", "assistants"]
)

text_data_2 = TextData(
    source="./data/documents/aoai-prompting.pdf",
    objective="Explains Prompting in Azure OpenAI",
    tags=["azure", "aoai", "prompting"]
)

print("TextData instance:", text_data_1)
print("TextData instance:", text_data_2)

TextData instance: source='./data/documents/aoai-assistants.pdf' objective='Explains Azure OpenAI and its features' tags=['azure', 'aoai', 'assistants'] encoding=None embeddings=None
TextData instance: source='./data/documents/aoai-prompting.pdf' objective='Explains Prompting in Azure OpenAI' tags=['azure', 'aoai', 'prompting'] encoding=None embeddings=None


### Loading Toolers and Creating the Assembly

In this step, we focus on loading the appropriate toolers required for processing the data. Toolers are specialized components designed to handle specific tasks, such as extracting embeddings, encoding data, or performing analysis. Once the toolers are loaded, they are assembled into a cohesive workflow to ensure seamless processing of the data models (audio, video, image, and text).

The assembly of toolers ensures that each data type is processed using the most suitable tools, leveraging their unique capabilities to extract meaningful insights and achieve the desired objectives.

In [5]:
import uuid
from app.schemas.models import Agent, Assembly

audio_agent = Agent(
    id=str(uuid.uuid4()),
    name="AudioAgent",
    model_id="default",
    metaprompt="This agent handles audio processing and extraction tasks.",
    objective="audio"
)

text_agent = Agent(
    id=str(uuid.uuid4()),
    name="TextAgent",
    model_id="default",
    metaprompt="This agent analyzes textual information for semantic understanding.",
    objective="text"
)

assembly = Assembly(
    id=str(uuid.uuid4()),
    objective="multimodal processing using local data for architecture review on azure",
    agents=[audio_agent, text_agent],
    roles=["audio", "text"]
)

print("Assembly created with agents:", assembly)

Assembly created with agents: id='67334a99-baac-416f-9b18-78ff78ef6612' objective='multimodal processing using local data for architecture review on azure' agents=[Agent(id='1f88b3b6-2e36-4bad-9088-0624668c628e', name='AudioAgent', model_id='default', metaprompt='This agent handles audio processing and extraction tasks.', objective='audio'), Agent(id='a2b31831-773e-40af-b925-9b5ecb8460ae', name='TextAgent', model_id='default', metaprompt='This agent analyzes textual information for semantic understanding.', objective='text')] roles=['audio', 'text']


In [6]:
from app.agents.main import ToolerOrchestrator

orchestrator = ToolerOrchestrator()

In [16]:
response = await orchestrator.run_interaction(assembly=assembly, prompt="Explain how may I use Azure OpenAI to build a chatbot on AKS or ACA", strategy="llm")
flattened = [str(item) for sublist in response for item in (sublist if isinstance(sublist, list) else [sublist])]
print("Orchestrator response:", "\n".join(flattened))

Orchestrator response: Tools available:
• Text extraction and summarization tool

Selected tool: Text extraction and summarization tool

Reference used:
– Official Microsoft Azure documentation (Azure OpenAI, Azure Kubernetes Service (AKS), and Azure Container Apps (ACA))

Answer:
To build a chatbot using Azure OpenAI on either AKS or ACA, you essentially need to combine the power of Azure’s OpenAI service with a containerized application deployed to one of these managed container platforms. Here’s a high-level step‐by‐step guide outlining the process:

1. Provision the Azure OpenAI Service:
 • Sign in to the Azure portal and create an Azure OpenAI resource. You’ll receive API keys and an endpoint URL that your chatbot backend will use to make calls to the language model.
 • Review usage limits and pricing details while ensuring that the specific OpenAI model(s) (like GPT-3.5 or GPT-4 variants) needed for your chatbot are enabled.

2. Develop the Chatbot Application:
 • Create your cha

In [8]:
response = await orchestrator.run_interaction(
    assembly=assembly,
    prompt="""
        You have been given a few local documents and audios on the folder located at the directory 'C:\\Users\\ricar\\Github\\augumented-rag\\notebook\\data'
        and contains the subfolders 'documents' and 'audios'. I need a abstract detailing their content.
        Preciselly give the name of the Python Classes used as Tools on your answer.
    """,
    strategy="parallel"
)

flattened = [
    str(item)
    for sublist in response
    for item in (sublist if isinstance(sublist, list) else [sublist])
]

print("Orchestrator response:", "\n".join(flattened))

Orchestrator response: It seems that your request is empty or missing specific instructions. Please provide details about the audio content you want analyzed or edited, such as:

- The audio files you'd like me to work on (e.g., filenames or folder location).
- The specific request or changes you'd like made to the audio (e.g., transcription, speaker identification, embedding generation, noise reduction, edits, etc.).
- The desired output or any additional information regarding the task.

Once you provide this information, I'll proceed with analyzing and processing the audio accordingly!
It appears no files or data could be retrieved from the specified directories `C:\Users\ricar\Github\augumented-rag\notebook\data\documents` or `C:\Users\ricar\Github\augumented-rag\notebook\data\audios`. Please verify the directory paths or contents to ensure the folders contain the expected files. Let me know how you'd like to proceed!


In [9]:
#response = await orchestrator.run_interaction(assembly=assembly, prompt="Detail each information that I have to consider on Azure Container Instance to Run a multi-agentic RAG app", strategy="llm")
#flattened = [str(item) for sublist in response for item in (sublist if isinstance(sublist, list) else [sublist])]
#print("Orchestrator response:", "\n".join(flattened))